In [1]:
%pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.8/644.8 KB 7.8 MB/s eta 0:00:008.6 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 3.9 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 KB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 MB 23.9 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 KB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 45.7 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 25.8 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 35.0 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.5/502.5 KB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 KB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 26.9 MB/s eta 

In [2]:
# Creating reusable training function

""" Testing on Training Set"""
import glob
import math
from ultralytics import YOLO
import time
import shutil
import os

# Returns model and the time it took to load the model
def load_model(model_name='yolov8x.pt'):
    start_time = time.perf_counter()
    model = YOLO(model_name)
    end_time = time.perf_counter()
    return model, end_time - start_time

# Returns trained model, results of training, and time to train the model
def train_model(model, yaml_filename, epochs=100, batch=4, patience=50, save_period=-1):
    start_time = time.perf_counter()
    results = model.train(data=yaml_filename, epochs=epochs, imgsz=640, device=[0], patience=patience, batch=batch, save_period=save_period)
    end_time = time.perf_counter()
    model.export()
    return model, results, end_time - start_time

# Returns # images fully labeled correct, # images, # lights identified correct, # lights, time to predict all images
def predict_images(model, input_im_path, output_im_path, print_scores=False, verbose=False):
    start_time = time.perf_counter()

    # Loading images
    test_images = glob.glob(f"{input_im_path}/images/*")

    # For calculating accuracy
    light_all = 0 
    image_all = 0
    light_correct = 0
    image_correct = 0
    
    for image in test_images:
    
        # Updating image counts
        image_all += 1
    
        # Making prediction
        prediction = model(image, save=True, boxes=True, save_conf=True, imgsz=320, project=output_im_path, verbose=verbose)
        coords = prediction[0].boxes.xywh
    
        # Reading from corresponding text file
        real_objects = []
        with open(f"{input_im_path}/labels/{image.split('/')[-1][:-4]}.txt", 'r') as f:
            for line in f:
                if line == "\n": continue
                vals = line.split(' ')
                # print(vals)
                real_objects.append( ( float(vals[1]), float(vals[2]), float(vals[3]), float(vals[4]) ) )
                light_all += 1
    
        # Determining how many images match
        no_errors = True
        for c in coords:
            this_x = float(c[0])
            this_y = float(c[1])
            this_w = float(c[2])
            this_h = float(c[3])

            # Checking each predicted box against each row of the true data
            this_coord_match = False
            for line in real_objects:
                if abs(this_x / 1280 - line[0]) <= 0.05 and abs(this_y / 960 - line[1]) <= 0.05 and abs(this_w / 1280 - line[2]) <= 0.05 and abs(this_h / 960 - line[3]) <= 0.05:
                    light_correct += 1
                    real_objects.remove(line)
                    if real_objects == None: real_objects = []
                    this_coord_match = True
                    break
    
            if not this_coord_match: no_errors = False
    
        if no_errors and len(real_objects): image_correct += 1

    # Moving all prediction images into one folder
    all_folders = glob.glob(f"{output_im_path}/*")
    for f in all_folders:
        these_files = glob.glob(f"{f}/*")
        if len(these_files) > 0:
            # print(these_files[0].split('/')[-1])
            
            shutil.copy(these_files[0], f"{output_im_path}/{these_files[0].split('/')[-1]}") # Copying over file
            os.remove(these_files[0])
            os.rmdir(f)

    end_time = time.perf_counter()
    if print_scores:
        print(f"Image accuracy: {image_correct} / {image_all}")
        print(f"Light accuracy: {light_correct} / {light_all}")

    return image_correct, image_all, light_correct, light_all, end_time - start_time
        

In [35]:
## Loading and predicting with untrained models
m, load_time = load_model()

ic, ia, lc, la, predict_time = predict_images(m, "YOLO_data/DatasetComp_2000/train", "Dataset2000_Comp_UntrainedModel_Predictions")

print(f"Image accuracy: {ic} / {ia}")
print(f"Light accuracy: {lc} / {la}")

print(f"Load time: {load_time} seconds.")
print(f"Predict time: {predict_time} seconds.")

Results saved to Dataset2000_Comp_UntrainedModel_Predictions/predict
Results saved to Dataset2000_Comp_UntrainedModel_Predictions/predict2
Results saved to Dataset2000_Comp_UntrainedModel_Predictions/predict3
Results saved to Dataset2000_Comp_UntrainedModel_Predictions/predict4
Results saved to Dataset2000_Comp_UntrainedModel_Predictions/predict5


['2', '0.621875', '0.32083333333333336', '0.0125', '0.028125\n']
['2', '0.69765625', '0.365625', '0.0109375', '0.028125\n']
['2', '0.7796875', '0.03958333333333333', '0.04609375', '0.07916666666666666\n']
['2', '0.56328125', '0.36041666666666666', '0.0109375', '0.021875\n']
['2', '0.64375', '0.403125', '0.0140625', '0.021875\n']
['2', '0.40390625', '0.37916666666666665', '0.00546875', '0.0125\n']
['2', '0.54375', '0.3385416666666667', '0.009375', '0.01875\n']
['2', '0.609375', '0.37395833333333334', '0.00703125', '0.015625\n']
['2', '0.9265625', '0.21354166666666666', '0.0390625', '0.0875\n']


Results saved to Dataset2000_Comp_UntrainedModel_Predictions/predict6
Results saved to Dataset2000_Comp_UntrainedModel_Predictions/predict7
Results saved to Dataset2000_Comp_UntrainedModel_Predictions/predict8
Results saved to Dataset2000_Comp_UntrainedModel_Predictions/predict9


['2', '0.52734375', '0.340625', '0.0125', '0.021875\n']
['2', '0.5640625', '0.34375', '0.009375', '0.025\n']
['2', '0.6296875', '0.37395833333333334', '0.01484375', '0.0375\n']
['2', '0.871875', '0.140625', '0.021875', '0.075\n']
['3', '0.67421875', '0.13541666666666666', '0.0359375', '0.078125\n']
['2', '0.04921875', '0.35104166666666664', '0.009375', '0.03125\n']
['2', '0.4265625', '0.3625', '0.009375', '0.025\n']
['2', '0.4640625', '0.23333333333333334', '0.01796875', '0.040625\n']
['2', '0.5765625', '0.3416666666666667', '0.01640625', '0.034375\n']
['2', '0.71640625', '0.215625', '0.03046875', '0.06770833333333333\n']
['2', '0.87265625', '0.2604166666666667', '0.0328125', '0.078125\n']
['1', '0.57421875', '0.20729166666666668', '0.03984375', '0.075\n']


Results saved to Dataset2000_Comp_UntrainedModel_Predictions/predict10
Results saved to Dataset2000_Comp_UntrainedModel_Predictions/predict11
Results saved to Dataset2000_Comp_UntrainedModel_Predictions/predict12
Results saved to Dataset2000_Comp_UntrainedModel_Predictions/predict13
Results saved to Dataset2000_Comp_UntrainedModel_Predictions/predict14


...

['2', '0.73671875', '0.171875', '0.0375', '0.09375\n']
['2', '0.90078125', '0.21458333333333332', '0.046875', '0.11979166666666667\n']
['1', '0.58359375', '0.17604166666666668', '0.046875', '0.08854166666666667\n']
['5', '0.84296875', '0.08125', '0.07265625', '0.121875\n']
['0', '0.4515625', '0.2604166666666667', '0.0265625', '0.07291666666666667\n']
['0', '0.54140625', '0.26666666666666666', '0.02890625', '0.078125\n']
['0', '0.70625', '0.3416666666666667', '0.01875', '0.05625\n']
['2', '0.8125', '0.19791666666666666', '0.02578125', '0.07291666666666667\n']
['1', '0.62578125', '0.19895833333333332', '0.0328125', '0.06979166666666667\n']


Results saved to Dataset2000_Comp_UntrainedModel_Predictions/predict1993
Results saved to Dataset2000_Comp_UntrainedModel_Predictions/predict1994
Results saved to Dataset2000_Comp_UntrainedModel_Predictions/predict1995
Results saved to Dataset2000_Comp_UntrainedModel_Predictions/predict1996
Results saved to Dataset2000_Comp_UntrainedModel_Predictions/predict1997


['2', '0.51015625', '0.31875', '0.0140625', '0.03125\n']
['2', '0.640625', '0.4010416666666667', '0.009375', '0.026041666666666668\n']
['1', '0.1', '0.4552083333333333', '0.03984375', '0.06979166666666667\n']
['2', '0.5390625', '0.3645833333333333', '0.0078125', '0.015625\n']
['0', '0.61328125', '0.06666666666666667', '0.046875', '0.08958333333333333\n']
['0', '0.93984375', '0.24895833333333334', '0.040625', '0.10625\n']
['0', '0.60703125', '0.07916666666666666', '0.0328125', '0.078125\n']
['0', '0.91796875', '0.24479166666666666', '0.0375', '0.07291666666666667\n']
['0', '0.2359375', '0.35208333333333336', '0.0140625', '0.028125\n']
['0', '0.27578125', '0.3729166666666667', '0.00546875', '0.010416666666666666\n']


Results saved to Dataset2000_Comp_UntrainedModel_Predictions/predict1998
Results saved to Dataset2000_Comp_UntrainedModel_Predictions/predict1999
Results saved to Dataset2000_Comp_UntrainedModel_Predictions/predict2000


['0', '0.6671875', '0.21770833333333334', '0.02578125', '0.06041666666666667\n']
['0', '0.78125', '0.246875', '0.02109375', '0.046875\n']
['0', '0.91484375', '0.328125', '0.02109375', '0.041666666666666664\n']
['0', '0.71953125', '0.20625', '0.0234375', '0.041666666666666664\n']
['0', '0.8734375', '0.2552083333333333', '0.01875', '0.041666666666666664\n']
['1', '0.5765625', '0.20104166666666667', '0.0421875', '0.065625\n']
['0', '0.45546875', '0.28541666666666665', '0.021875', '0.05625\n']
['0', '0.5265625', '0.28854166666666664', '0.021875', '0.06145833333333333\n']
['0', '0.6515625', '0.346875', '0.01484375', '0.041666666666666664\n']
night_699.jpg
night_665.jpg
day_275.jpg
night_134.jpg
day_223.jpg
day_226.jpg
day_509.jpg
night_814.jpg
night_384.jpg
night_570.jpg
night_457.jpg
night_223.jpg
day_870.jpg
day_192.jpg
night_303.jpg
day_440.jpg
day_273.jpg
day_1000.jpg
night_789.jpg
night_83.jpg
night_394.jpg
day_995.jpg
night_956.jpg
day_283.jpg
day_859.jpg
night_401.jpg
night_533.jpg
n

In [5]:
""" Experiment: what is the effect of the number of epochs on training accuracy """

# DRAWING FROM PRE-LOADED MODELS

m, load_time = load_model()

m, results, train_time = train_model(m, "Dataset1_2000.yaml", epochs=2000, save_period = 50)

""" Predicting Later """

New https://pypi.org/project/ultralytics/8.0.210 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.203 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11003MiB)
engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=Dataset1_2000.yaml, epochs=2000, patience=50, batch=4, imgsz=640, save=True, save_period=50, cache=False, device=[0], workers=8, project=None, name=train27, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augm

' Predicting Later '

In [9]:
m, load_time = load_model("runs_keep/231115_2000_250_Epochs/weights/best.pt")
print("HERE")
dic, dia, dlc, dla, dpredict_time = predict_images(m, "YOLO_data/DatasetComp_1000Day", "Predictions/Dataset2000_250_Epoch_Predictions")
nic, nia, nlc, nla, npredict_time = predict_images(m, "YOLO_data/DatasetComp_1000Night", "Predictions/Dataset2000_250_Epoch_Predictions")

HERE


Results saved to Predictions/Dataset2000_250_Epoch_Predictions/predict
Results saved to Predictions/Dataset2000_250_Epoch_Predictions/predict2
Results saved to Predictions/Dataset2000_250_Epoch_Predictions/predict3
Results saved to Predictions/Dataset2000_250_Epoch_Predictions/predict4
Results saved to Predictions/Dataset2000_250_Epoch_Predictions/predict5
Results saved to Predictions/Dataset2000_250_Epoch_Predictions/predict6
Results saved to Predictions/Dataset2000_250_Epoch_Predictions/predict7
Results saved to Predictions/Dataset2000_250_Epoch_Predictions/predict8
Results saved to Predictions/Dataset2000_250_Epoch_Predictions/predict9
Results saved to Predictions/Dataset2000_250_Epoch_Predictions/predict10
Results saved to Predictions/Dataset2000_250_Epoch_Predictions/predict11
Results saved to Predictions/Dataset2000_250_Epoch_Predictions/predict12
Results saved to Predictions/Dataset2000_250_Epoch_Predictions/predict13
Results saved to Predictions/Dataset2000_250_Epoch_Prediction

In [11]:
# Writing output to a file
with open("Experiment3_Epochs.txt", 'a') as f:
    f.write(f"Day Image accuracy: {dic} / {dia}, or {(100 * dic / dia):.2f}%.\n")
    f.write(f"Day Light accuracy: {dlc} / {dla}, or {(100 * dlc / dla):.2f}%.\n")
    f.write(f"Night Image accuracy: {nic} / {nia}, or {(100 * nic / nia):.2f}%.\n")
    f.write(f"Night Light accuracy: {nlc} / {nla}, or {(100 * nlc / nla):.2f}%.\n")